### Recommendations Rank & Knowledge Based: 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')
del movies['Unnamed: 0']
del reviews['Unnamed: 0']

In [4]:
reviews.columns

Index(['user_id', 'movie_id', 'rating', 'timestamp', 'date', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'year_2013',
       'year_2014', 'year_2015', 'year_2016', 'year_2017', 'year_2018'],
      dtype='object')

In [7]:
movies.head(5)

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [5]:
# merged_data = pd.merge(movies,reviews,how='left',on='movie_id')
# merged_data.columns

#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [8]:
reivews_grouped = reviews.groupby('movie_id').agg({'rating':['mean','count'],'date':['max']})
reivews_grouped.columns=['rating_mean','rating_count','max_date']
reivews_grouped=reivews_grouped.reset_index()

In [9]:
reivews_grouped = reviews.groupby('movie_id').agg({'rating':['mean','count'],'date':['max']})
reivews_grouped.columns=['rating_mean','rating_count','max_date']
reivews_grouped=reivews_grouped.reset_index()
sorted_grouped_movies = reivews_grouped.sort_values(by=['rating_mean','rating_count','max_date'],ascending=False)
sorted_grouped_movies.query('rating_count>=20')

,movie_id,rating_mean,rating_count,max_date
28837,4921860,10.000000,48,2016-08-14 17:16:50
29398,5262972,10.000000,28,2016-01-08 00:44:43
29679,5512872,9.985836,353,2018-05-17 00:29:18
8049,111161,9.407809,922,2018-06-26 14:15:09
30056,5813916,9.382353,34,2018-02-22 19:24:37
...,...,...,...,...
18207,1540767,2.434783,23,2016-09-19 17:37:17
30239,5988370,2.285714,35,2017-03-19 15:13:36
14814,829176,2.269231,26,2018-02-07 04:59:11
21556,2147365,2.142857,21,2015-02-01 00:52:05


In [6]:
reviews.query('movie_id == 8342946')

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
356595,26919,8342946,2,1529291786,2018-06-18 03:16:26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
561922,42089,8342946,7,1527907556,2018-06-02 02:45:56,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [45]:
def popular_recommendations(user_id, n_top,min_num_rating=5):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    
    # Do stuff first try with wrong try to improve it , because it is ver complicated 
    # sorted_by_timestamp_data = merged_data.sort_values(by='date_x',ascending=False)
    # filtered_by_mean_count_rating_data =  sorted_by_timestamp_data.loc[sorted_by_timestamp_data['rating']>=5].\
    #                                         groupby('movie').\
    #                                         agg(mean_rating=pd.NamedAgg(column='rating',aggfunc='mean')
    #                                             ,count_rating=pd.NamedAgg(column='rating',aggfunc='count')).\
    #                                         reset_index()                                           
    # sorted_by_rating_data = filtered_by_mean_count_rating_data.sort_values(by=['count_rating','mean_rating'],ascending=[False,False])
    # top_movies = sorted_by_rating_data.iloc[0:n_top,:]['movie']

    ##################Another Solution ############################
    reivews_grouped = reviews.groupby('movie_id').agg({'rating':['mean','count'],'date':['max']})
    reivews_grouped.columns=['rating_mean','rating_count','max_date']
    reivews_grouped=reivews_grouped.reset_index()
    sorted_grouped_movies = reivews_grouped.sort_values(by=['rating_mean','rating_count','max_date'],ascending=False)
    top_movies_ids = list(sorted_grouped_movies[sorted_grouped_movies['rating_count']>=min_num_rating]['movie_id'])
    #top_movies_titles = list(movies[movies['movie_id'].isin(top_movies_ids)]['movie'])
    top_movies_titles = list(movies[movies['movie_id'].isin(top_movies_ids)]['movie'][:n_top])
    #TODO return titles of films  
    return top_movies_titles
    ################# Submitted Solution ###########################
    #my problem is to merge at first but it get my dataframes to be more complicated without nesscessry
    # movie_ratings = reviews.groupby('movie_id')['rating']
    # avg_ratings = movie_ratings.mean()
    # count_ratings = movie_ratings.count()
    # last_rating = pd.DataFrame(reviews.groupby('movie_id').max()['date'])
    # last_rating.columns = ['last_rating']

    # rating_count_df = pd.DataFrame({'avg_ratings':avg_ratings,'count_ratings':count_ratings})
    # rating_count_df = rating_count_df.join(last_rating)

    # movie_recs = movies.set_index('movie_id').join(rating_count_df)

    # # sort by top avg rating and number of ratings
    # ranked_movies = movie_recs.sort_values(['avg_ratings', 'count_ratings', 'last_rating'], ascending=False)

    # # for edge cases - subset the movie list to those with only 5 or more reviews
    # ranked_movies = ranked_movies[ranked_movies['count_ratings'] > 4]

    # return list(ranked_movies['movie'].iloc[:n_top]) # a list of the n_top movies as recommended

In [46]:
popular_recommendations(26919,10)

['Le voyage dans la lune (1902)',
 "Intolerance: Love's Struggle Throughout the Ages (1916)",
 'Das Cabinet des Dr. Caligari (1920)',
 'The Kid (1921)',
 'Körkarlen (1921)',
 'Häxan (1922)',
 'Nanook of the North (1922)',
 'Nosferatu, eine Symphonie des Grauens (1922)',
 'Our Hospitality (1923)',
 'Safety Last! (1923)']

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [47]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = popular_recommendations(1,20)# Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = popular_recommendations(53968,5)# Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = popular_recommendations(70000,100)# Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = popular_recommendations(43,35)# Your solution list here



In [44]:
print(len(recs_20_for_1))

9974


In [177]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

If you got here, looks like you are good to go!  Nice job!


**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History', 'Comedy'])
```

In [69]:
def popular_recs_filtered(user_id,n_top,ranked_movies,years=None,genres=None):
     # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]


    # create top movies list
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies

In [49]:
ranked_movies_titles = popular_recommendations(1,20,7)

In [52]:
print(len(ranked_movies_titles))
ranked_movies = movies[movies['movie'].isin(ranked_movies_titles)]
ranked_movies

20


,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
5,417,Le voyage dans la lune (1902),Short|Adventure|Fantasy,1902,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
55,10323,Das Cabinet des Dr. Caligari (1920),Horror|Mystery,1920,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
70,12349,The Kid (1921),Comedy|Drama|Family,1921,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
71,12364,Körkarlen (1921),Drama|Fantasy|Horror,1921,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
80,13257,Häxan (1922),Documentary|Fantasy|Horror,1922,0,1,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
81,13427,Nanook of the North (1922),Documentary,1922,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
82,13442,"Nosferatu, eine Symphonie des Grauens (1922)",Horror,1922,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
88,14341,Our Hospitality (1923),Comedy|Thriller,1923,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
91,14429,Safety Last! (1923),Comedy|Thriller,1923,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
99,15064,Der letzte Mann (1924),Drama,1924,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
popular_filtered = popular_recs_filtered('1', 20, ranked_movies, years=[1926,1927],genres=['Comedy','Drama','Horror','Comedy'])
popular_filtered

['Faust - Eine deutsche Volkssage (1926)',
 'The Lodger: A Story of the London Fog (1927)',
 'Metropolis (1927)',
 'The General (1926)',
 'Sunrise: A Song of Two Humans (1927)',
 'Wings (1927)']

In [64]:
assert t.popular_recs_filtered('1', 20, ranked_movies, years=[2015, 2016, 2017, 2018], genres=['History', 'Comedy']) == popular_filtered,  "The popular check failed..."
print("If you got here, looks like you are good to go!  Nice job!")

# Amazing End